# Calculate distance from NHP

This notebook helps calculate the distance from a patient's residental area to National Hospital of Pediatrics, Hanoi, Vietnam, using `Azure Maps Route` and `Geopy` libraries.

In [ ]:
# Import necessary libraries
from azure.maps.route import MapsRouteClient
from azure.core.credentials import AzureKeyCredential
from azure.maps.route.models import LatLon
from geopy.geocoders import AzureMaps
from pprint import pprint

In [ ]:
# Create map and route clients
credential = AzureKeyCredential('AZURE-MAPS-KEY')
geolocation_client = AzureMaps(credential.key)
route_client = MapsRouteClient(credential=credential)

In [ ]:
# Set default location for NHP to prevent miscalculation
NHP_coordinates = LatLon(21.026047, 105.810077)
print('NHP coordinates:', geolocation_client.reverse("%f, %f" % (NHP_coordinates.lat, NHP_coordinates.lon)))

In [ ]:
# Example
community = "Thi tran Cho"
district = "Huyen Yen Phong"
province = "Bac Ninh"

In [ ]:
# Get location
location = geolocation_client.geocode(community + ", " + district + ", " + province + ", " + "Vietnam")
print('Location:', location, location.latitude, location.longitude)
route = route_client.get_route_directions(route_points=[
    LatLon(location.latitude, location.longitude),
    NHP_coordinates
])

# Print output summary
# The output contains the distance in meters and travel time in seconds
pprint(route.as_dict()['routes'][0]['summary'])  